In [1]:
%matplotlib inline
import pandas as pd
import itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import bnpy
import warnings
from scipy import stats
import random

from scipy.stats import kruskal, f_oneway
import collections
import scikit_posthocs as sp

warnings.filterwarnings('ignore')

In [2]:
pth = '../data/expression/target-high-risk-nbl-mycn-na-exp-2018-11-12.tsv'

exp = pd.read_csv(pth, sep='\t', index_col=0) 

In [3]:
pth = '../data/archive/mycn-na-nbl-0.2-assign-2019-03-02.tsv'
assign = pd.read_csv(pth, sep='\t', index_col=0)

In [4]:
assign

,1
TARGET-30-PAITCI-01,3
TARGET-30-PAITEG-01,3
TARGET-30-PAIVHE-01,3
TARGET-30-PALBFW-01,3
TARGET-30-PALCBW-01,3
TARGET-30-PALETP-01,1
TARGET-30-PALIIN-01,1
TARGET-30-PALNVP-01,2
TARGET-30-PALTEG-01,1
TARGET-30-PALUYS-01,2


In [25]:
_dir = '../data/gsea/'

sub_cohort = collections.defaultdict(list)
sub_cohort_unique = collections.defaultdict(list)
sub_clust_unique = collections.defaultdict(list)

interest = ['HUMORAL IMMUNE RESPONSE MEDIATED BY CIRCULATING IMMUNOGLOBULIN%GOBP%GO:0002455',
 'COMPLEMENT ACTIVATION, CLASSICAL PATHWAY%GOBP%GO:0006958',
 'IMMUNOGLOBULIN COMPLEX%GOCC%GO:0019814',
 'LEUKOCYTE DEGRANULATION%GOBP%GO:0043299',
 'MYELOID LEUKOCYTE MEDIATED IMMUNITY%GOBP%GO:0002444',
 'IMMUNOGLOBULIN RECEPTOR BINDING%GOMF%GO:0034987',
 'GRANULOCYTE ACTIVATION%GOBP%GO:0036230',
 'INFLAMMATORY RESPONSE%GOBP%GO:0006954',
 'CELL ACTIVATION INVOLVED IN IMMUNE RESPONSE%GOBP%GO:0002263',
 'NEUTROPHIL ACTIVATION%GOBP%GO:0042119']

cluster2_immune = []
for cluster, rows in assign.groupby('1'):
    samps = rows.index.values
    for samp in samps:
        pth = os.path.join(_dir, 'sub_cohort', str(cluster), samp)        
        f1 = pd.read_csv(pth, index_col=0)
        f1 = f1[(f1['padj'] < 0.05) & (f1['ES'] > 0.0)]
        
        pth = os.path.join(_dir, 'sub_clust', str(cluster), samp)
        f2 = pd.read_csv(pth, index_col=0)
        f2 = f2[(f2['padj'] < 0.05) & (f2['ES'] > 0.0)]
        
        if cluster == 2:
            print samp
            print f1.sort_values('NES', ascending=False)['NES'].head(10)
            print '\n'
            print f2.sort_values('NES', ascending=False)['NES'].head(50)
            print '\n\n'
            
            if len(f2.index.intersection(interest)) > 0:
                cluster2_immune.append(samp)
        
        sub_cohort_unique[cluster].extend(list(set(f1.index.values) - set(f2.index.values)))
        sub_clust_unique[cluster].extend(list(set(f2.index.values) - set(f1.index.values)))

TARGET-30-PALNVP-01
pathway
HALLMARK_OXIDATIVE_PHOSPHORYLATION%MSIGDB_C2%HALLMARK_OXIDATIVE_PHOSPHORYLATION                                                                4.487746
MITOCHONDRIAL PROTEIN COMPLEX%GOCC%GO:0098798                                                                                                  4.174388
RESPIRATORY ELECTRON TRANSPORT, ATP SYNTHESIS BY CHEMIOSMOTIC COUPLING, AND HEAT PRODUCTION BY UNCOUPLING PROTEINS.%REACTOME%R-HSA-163200.1    4.166585
INNER MITOCHONDRIAL MEMBRANE PROTEIN COMPLEX%GOCC%GO:0098800                                                                                   4.133840
ATP METABOLIC PROCESS%GOBP%GO:0046034                                                                                                          4.067417
OXIDATIVE PHOSPHORYLATION%GOBP%GO:0006119                                                                                                      4.023577
NUCLEOSIDE MONOPHOSPHATE METABOLIC PROCESS%GOBP%GO:0009123  

TARGET-30-PALXTB-01
pathway
EUKARYOTIC TRANSLATION ELONGATION%REACTOME DATABASE ID RELEASE 66%156842                4.113135
PEPTIDE CHAIN ELONGATION%REACTOME DATABASE ID RELEASE 66%156902                         4.006488
VIRAL MRNA TRANSLATION%REACTOME%R-HSA-192823.3                                          3.883829
CYTOSOLIC LARGE RIBOSOMAL SUBUNIT%GOCC%GO:0022625                                       3.859249
SRP-DEPENDENT COTRANSLATIONAL PROTEIN TARGETING TO MEMBRANE%REACTOME%R-HSA-1799339.1    3.828055
FORMATION OF A POOL OF FREE 40S SUBUNITS%REACTOME DATABASE ID RELEASE 66%72689          3.780778
SELENOCYSTEINE SYNTHESIS%REACTOME%R-HSA-2408557.2                                       3.718229
SRP-DEPENDENT COTRANSLATIONAL PROTEIN TARGETING TO MEMBRANE%GOBP%GO:0006614             3.610826
COTRANSLATIONAL PROTEIN TARGETING TO MEMBRANE%GOBP%GO:0006613                           3.593486
EUKARYOTIC TRANSLATION TERMINATION%REACTOME%R-HSA-72764.4                               3.544738
Na

TARGET-30-PANBSP-01
pathway
MAJOR PATHWAY OF RRNA PROCESSING IN THE NUCLEOLUS AND CYTOSOL%REACTOME DATABASE ID RELEASE 66%6791226       4.420379
RRNA PROCESSING IN THE NUCLEUS AND CYTOSOL%REACTOME%R-HSA-8868773.2                                         4.405973
L13A-MEDIATED TRANSLATIONAL SILENCING OF CERULOPLASMIN EXPRESSION%REACTOME DATABASE ID RELEASE 66%156827    4.401485
CAP-DEPENDENT TRANSLATION INITIATION%REACTOME DATABASE ID RELEASE 66%72737                                  4.398666
EUKARYOTIC TRANSLATION INITIATION%REACTOME DATABASE ID RELEASE 66%72613                                     4.398666
RRNA PROCESSING%REACTOME DATABASE ID RELEASE 66%72312                                                       4.397193
TRANSLATIONAL INITIATION%GOBP%GO:0006413                                                                    4.396977
GTP HYDROLYSIS AND JOINING OF THE 60S RIBOSOMAL SUBUNIT%REACTOME DATABASE ID RELEASE 66%72706               4.380638
FORMATION OF A POOL OF FREE 40S SUBU

TARGET-30-PANWRR-01
pathway
EUKARYOTIC TRANSLATION TERMINATION%REACTOME%R-HSA-72764.4                                                              3.392484
RRNA PROCESSING IN THE NUCLEUS AND CYTOSOL%REACTOME%R-HSA-8868773.2                                                    3.359649
MAJOR PATHWAY OF RRNA PROCESSING IN THE NUCLEOLUS AND CYTOSOL%REACTOME DATABASE ID RELEASE 66%6791226                  3.348044
NONSENSE MEDIATED DECAY (NMD) INDEPENDENT OF THE EXON JUNCTION COMPLEX (EJC)%REACTOME DATABASE ID RELEASE 66%975956    3.304817
FORMATION OF A POOL OF FREE 40S SUBUNITS%REACTOME DATABASE ID RELEASE 66%72689                                         3.292014
RRNA PROCESSING%REACTOME DATABASE ID RELEASE 66%72312                                                                  3.268751
L13A-MEDIATED TRANSLATIONAL SILENCING OF CERULOPLASMIN EXPRESSION%REACTOME DATABASE ID RELEASE 66%156827               3.252640
CYTOSOLIC RIBOSOME%GOCC%GO:0022626                                          

TARGET-30-PAPBGH-01
pathway
HALLMARK_MYC_TARGETS_V1%MSIGDB_C2%HALLMARK_MYC_TARGETS_V1                                                   4.404685
L13A-MEDIATED TRANSLATIONAL SILENCING OF CERULOPLASMIN EXPRESSION%REACTOME DATABASE ID RELEASE 66%156827    4.384208
TRANSLATION%REACTOME%R-HSA-72766.3                                                                          4.364429
GTP HYDROLYSIS AND JOINING OF THE 60S RIBOSOMAL SUBUNIT%REACTOME DATABASE ID RELEASE 66%72706               4.358441
RRNA PROCESSING%REACTOME DATABASE ID RELEASE 66%72312                                                       4.339856
RRNA PROCESSING IN THE NUCLEUS AND CYTOSOL%REACTOME%R-HSA-8868773.2                                         4.325765
EUKARYOTIC TRANSLATION INITIATION%REACTOME DATABASE ID RELEASE 66%72613                                     4.301435
CAP-DEPENDENT TRANSLATION INITIATION%REACTOME DATABASE ID RELEASE 66%72737                                  4.301435
TRANSLATION%GOBP%GO:0006412         

...

TARGET-30-PARHAM-01
pathway
AXON%GOCC%GO:0030424                                               3.033915
CHOLESTEROL BIOSYNTHETIC PROCESS%GOBP%GO:0006695                   2.886191
TRANS-SYNAPTIC SIGNALING%GOBP%GO:0099537                           2.882866
CHOLESTEROL BIOSYNTHESIS%REACTOME DATABASE ID RELEASE 66%191273    2.880702
SYNAPTIC SIGNALING%GOBP%GO:0099536                                 2.872223
SECONDARY ALCOHOL BIOSYNTHETIC PROCESS%GOBP%GO:1902653             2.857894
CHEMICAL SYNAPTIC TRANSMISSION%GOBP%GO:0007268                     2.857866
ANTEROGRADE TRANS-SYNAPTIC SIGNALING%GOBP%GO:0098916               2.857866
SUPERPATHWAY OF CHOLESTEROL BIOSYNTHESIS%HUMANCYC%PWY66-5          2.851426
MAIN AXON%GOCC%GO:0044304                                          2.834517
Name: NES, dtype: float64


pathway
EXTRACELLULAR MATRIX STRUCTURAL CONSTITUENT%GOMF%GO:0005201                                                                                                                     

TARGET-30-PASPER-01
pathway
FORMATION OF A POOL OF FREE 40S SUBUNITS%REACTOME DATABASE ID RELEASE 66%72689                                         2.923883
L13A-MEDIATED TRANSLATIONAL SILENCING OF CERULOPLASMIN EXPRESSION%REACTOME DATABASE ID RELEASE 66%156827               2.885781
TRANSLATIONAL INITIATION%GOBP%GO:0006413                                                                               2.867049
GTP HYDROLYSIS AND JOINING OF THE 60S RIBOSOMAL SUBUNIT%REACTOME DATABASE ID RELEASE 66%72706                          2.854142
PEPTIDE CHAIN ELONGATION%REACTOME DATABASE ID RELEASE 66%156902                                                        2.789834
VIRAL MRNA TRANSLATION%REACTOME%R-HSA-192823.3                                                                         2.788737
EUKARYOTIC TRANSLATION INITIATION%REACTOME DATABASE ID RELEASE 66%72613                                                2.775337
CAP-DEPENDENT TRANSLATION INITIATION%REACTOME DATABASE ID RELEASE 66%72737  

TARGET-30-PASUCB-01
pathway
HALLMARK_MYC_TARGETS_V1%MSIGDB_C2%HALLMARK_MYC_TARGETS_V1                                                3.579836
MAJOR PATHWAY OF RRNA PROCESSING IN THE NUCLEOLUS AND CYTOSOL%REACTOME DATABASE ID RELEASE 66%6791226    3.463813
RRNA PROCESSING IN THE NUCLEUS AND CYTOSOL%REACTOME%R-HSA-8868773.2                                      3.460098
TRANSLATION%GOBP%GO:0006412                                                                              3.420247
STRUCTURAL CONSTITUENT OF RIBOSOME%GOMF%GO:0003735                                                       3.370185
CYTOSOLIC LARGE RIBOSOMAL SUBUNIT%GOCC%GO:0022625                                                        3.364298
RRNA PROCESSING%REACTOME DATABASE ID RELEASE 66%72312                                                    3.360320
LARGE RIBOSOMAL SUBUNIT%GOCC%GO:0015934                                                                  3.346262
RIBOSOMAL SUBUNIT%GOCC%GO:0044391                           

TARGET-30-PATCFL-01
pathway
CYTOSOLIC LARGE RIBOSOMAL SUBUNIT%GOCC%GO:0022625                                                                      3.548190
FORMATION OF A POOL OF FREE 40S SUBUNITS%REACTOME DATABASE ID RELEASE 66%72689                                         3.377231
EUKARYOTIC TRANSLATION ELONGATION%REACTOME DATABASE ID RELEASE 66%156842                                               3.354453
PEPTIDE CHAIN ELONGATION%REACTOME DATABASE ID RELEASE 66%156902                                                        3.252655
EUKARYOTIC TRANSLATION TERMINATION%REACTOME%R-HSA-72764.4                                                              3.215463
SELENOCYSTEINE SYNTHESIS%REACTOME%R-HSA-2408557.2                                                                      3.179584
VIRAL MRNA TRANSLATION%REACTOME%R-HSA-192823.3                                                                         3.159085
SRP-DEPENDENT COTRANSLATIONAL PROTEIN TARGETING TO MEMBRANE%GOBP%GO:0006614 

TARGET-30-PATYIL-01
pathway
TRANSLATION%GOBP%GO:0006412                                                                                 3.607377
TRANSLATION%REACTOME%R-HSA-72766.3                                                                          3.563803
PEPTIDE BIOSYNTHETIC PROCESS%GOBP%GO:0043043                                                                3.524271
PYRUVATE METABOLIC PROCESS%GOBP%GO:0006090                                                                  3.484642
AMIDE BIOSYNTHETIC PROCESS%GOBP%GO:0043604                                                                  3.474870
HALLMARK_MYC_TARGETS_V1%MSIGDB_C2%HALLMARK_MYC_TARGETS_V1                                                   3.452851
EUKARYOTIC TRANSLATION INITIATION%REACTOME DATABASE ID RELEASE 66%72613                                     3.402138
CAP-DEPENDENT TRANSLATION INITIATION%REACTOME DATABASE ID RELEASE 66%72737                                  3.402138
TRANSLATIONAL INITIATION%GOBP%GO:000

In [26]:
len(cluster2_immune)

14

In [27]:
for cluster, counts in sub_cohort_unique.items():
    print cluster, len(assign[assign['1'] == cluster])
    
    counter = collections.Counter(counts)
    
    for gs, n in counter.most_common(10):
        print gs, n
    
    print '\n\n'

1 31
POSITIVE T CELL SELECTION%GOBP%GO:0043368 23
THYMIC T CELL SELECTION%GOBP%GO:0045061 23
ALPHA-BETA T CELL RECEPTOR COMPLEX%GOCC%GO:0042105 22
POSITIVE THYMIC T CELL SELECTION%GOBP%GO:0045059 22
T CELL RECEPTOR COMPLEX%GOCC%GO:0042101 21
T CELL DIFFERENTIATION%GOBP%GO:0030217 20
T CELL SELECTION%GOBP%GO:0045058 19
ESTABLISHMENT OF T CELL POLARITY%GOBP%GO:0001768 19
REGULATION OF DENDRITIC CELL DENDRITE ASSEMBLY%GOBP%GO:2000547 19
BIOCARTA_TCRA_PATHWAY%MSIGDB_C2%BIOCARTA_TCRA_PATHWAY 18



2 26
TRNA CHARGING%HUMANCYC%TRNA-CHARGING-PWY 11
MITOCHONDRIAL TRANSLATION ELONGATION%REACTOME DATABASE ID RELEASE 66%5389840 11
MITOCHONDRIAL TRANSLATION%REACTOME DATABASE ID RELEASE 66%5368287 11
MITOCHONDRIAL MEMBRANE PART%GOCC%GO:0044455 11
MITOCHONDRIAL TRANSLATIONAL ELONGATION%GOBP%GO:0070125 11
MITOCHONDRIAL TRANSLATION TERMINATION%REACTOME DATABASE ID RELEASE 66%5419276 11
MITOCHONDRIAL TRANSLATION%GOBP%GO:0032543 11
TRNA AMINOACYLATION%REACTOME DATABASE ID RELEASE 66%379724 11
MITOCHONDRI

In [28]:
top5 = []
for cluster, counts in sub_clust_unique.items():
    print cluster, len(assign[assign['1'] == cluster])
    
    counter = collections.Counter(counts)
    
    for gs, n in counter.most_common(10):
        print gs, n
        if cluster == 2:
            top5.append(gs)
    
    print '\n\n'

1 31
ADHERENS JUNCTION%GOCC%GO:0005912 10
ANCHORING JUNCTION%GOCC%GO:0070161 10
CELL-SUBSTRATE JUNCTION%GOCC%GO:0030055 9
CELL-SUBSTRATE ADHERENS JUNCTION%GOCC%GO:0005924 8
EXTRACELLULAR MATRIX%GOCC%GO:0031012 8
REGULATION OF VASCULATURE DEVELOPMENT%GOBP%GO:1901342 8
FOCAL ADHESION%GOCC%GO:0005925 8
POSITIVE REGULATION OF LOCOMOTION%GOBP%GO:0040017 8
POSITIVE REGULATION OF CELL MIGRATION%GOBP%GO:0030335 8
POSITIVE REGULATION OF CELL MOTILITY%GOBP%GO:2000147 8



2 26
HUMORAL IMMUNE RESPONSE MEDIATED BY CIRCULATING IMMUNOGLOBULIN%GOBP%GO:0002455 8
COMPLEMENT ACTIVATION, CLASSICAL PATHWAY%GOBP%GO:0006958 8
IMMUNOGLOBULIN COMPLEX%GOCC%GO:0019814 8
LEUKOCYTE DEGRANULATION%GOBP%GO:0043299 8
MYELOID LEUKOCYTE MEDIATED IMMUNITY%GOBP%GO:0002444 8
IMMUNOGLOBULIN RECEPTOR BINDING%GOMF%GO:0034987 7
GRANULOCYTE ACTIVATION%GOBP%GO:0036230 7
INFLAMMATORY RESPONSE%GOBP%GO:0006954 7
CELL ACTIVATION INVOLVED IN IMMUNE RESPONSE%GOBP%GO:0002263 7
NEUTROPHIL ACTIVATION%GOBP%GO:0042119 7



3 13
CONDENSED 

In [29]:
top5

['HUMORAL IMMUNE RESPONSE MEDIATED BY CIRCULATING IMMUNOGLOBULIN%GOBP%GO:0002455',
 'COMPLEMENT ACTIVATION, CLASSICAL PATHWAY%GOBP%GO:0006958',
 'IMMUNOGLOBULIN COMPLEX%GOCC%GO:0019814',
 'LEUKOCYTE DEGRANULATION%GOBP%GO:0043299',
 'MYELOID LEUKOCYTE MEDIATED IMMUNITY%GOBP%GO:0002444',
 'IMMUNOGLOBULIN RECEPTOR BINDING%GOMF%GO:0034987',
 'GRANULOCYTE ACTIVATION%GOBP%GO:0036230',
 'INFLAMMATORY RESPONSE%GOBP%GO:0006954',
 'CELL ACTIVATION INVOLVED IN IMMUNE RESPONSE%GOBP%GO:0002263',
 'NEUTROPHIL ACTIVATION%GOBP%GO:0042119']

In [30]:
def get_event(event):
    if pd.isnull(event):
        print("NULL")
        return np.nan
    
    events = ['Relapse',
              'Death', 
              'Progression',
              'Event',
              'Second Malignant Neoplasm']
    
    if event == 'Censored':
        return 0
    
    elif event in events:
        return 1 
    
    else:
        raise ValueError(event)
        
def get_vital(vital):
    if pd.isnull(vital):
        print("NULL")
        return np.nan
    
    
    if vital == 'Alive':
        return 0
    
    elif vital == 'Dead':
        return 1
    
    else:
        raise ValueError(vital)

pth = '../data/meta/TARGET_NBL_ClinicalData_Discovery_20170525.xlsx'
clinical = pd.read_excel(pth, index_col=0)

surv = pd.DataFrame(index=assign[assign['1'] == 2].index, 
                    columns=['OS', 'vital', 'EFS', 'event', 'immune'])

for sample in assign[assign['1'] == 2].index:
    root = '-'.join(sample.split('-')[:3])
    if root not in clinical.index:
        print "Missing: ", sample
        continue
    
    OS = clinical.loc[root, 'Overall Survival Time in Days'].item()
    vital = get_vital(clinical.loc[root, 'Vital Status'])
    
    EFS = clinical.loc[root, 'Event Free Survival Time in Days'].item()
    event = get_event(clinical.loc[root, 'First Event'])
    
    
    immune = 1 if sample in cluster2_immune else 0
    surv.loc[sample, :] = [OS, vital, EFS, event, immune]

In [31]:
import uuid

# Can't figure out how to pass this dataframe
survpth = '../data/mycn-na-tme-signal-subtract-cluster2-survival.tsv'
surv.to_csv(survpth, sep='\t')

In [32]:
cluster2_immune

['TARGET-30-PALNVP-01',
 'TARGET-30-PALUYS-01',
 'TARGET-30-PALXTB-01',
 'TARGET-30-PANUIF-01',
 'TARGET-30-PANWRR-01',
 'TARGET-30-PAPEAV-01',
 'TARGET-30-PAPLSD-01',
 'TARGET-30-PARGKK-01',
 'TARGET-30-PARHAM-01',
 'TARGET-30-PARHAM-02',
 'TARGET-30-PASPER-01',
 'TARGET-30-PASUYG-01',
 'TARGET-30-PATCFL-01',
 'TARGET-30-PATYIL-02']

In [23]:
og_list = ['TARGET-30-PALNVP-01',
 'TARGET-30-PALUYS-01',
 'TARGET-30-PALXTB-01',
 'TARGET-30-PANUIF-01',
 'TARGET-30-PAPEAV-01',
 'TARGET-30-PAPLSD-01',
 'TARGET-30-PARGKK-01',
 'TARGET-30-PARHAM-02',
 'TARGET-30-PASPER-01',
 'TARGET-30-PASUYG-01',
 'TARGET-30-PATCFL-01',
 'TARGET-30-PATYIL-02']

In [33]:
set(og_list) - set(cluster2_immune) 

set()